# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Importing Libraries and Modules</p>

In [1]:
import pandas as pd
import pandas as pd
import seaborn as sb
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,LabelEncoder,PolynomialFeatures
from sklearn.compose import make_column_selector,ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from lightgbm import LGBMRegressor

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Reading Dataset</p>

In [2]:
data=pd.read_csv('/kaggle/input/sample34/processes2.csv').drop('Unnamed: 0',axis=1)

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Machine Learning</p>

In [3]:
one=('oh',OneHotEncoder(handle_unknown='ignore',sparse=False),['name','fuel','seller_type','transmission','seats','owner','Mileage Unit','year'])
ss=('ss',StandardScaler(),['km_driven','Mileage','Engine (CC)'])
mm=('mm',MinMaxScaler(),['km_driven','Mileage','Engine (CC)'])

In [4]:
trans=ColumnTransformer([one,ss,mm],remainder='passthrough')

In [5]:
x=data.drop('selling_price',axis=1)
y=data['selling_price']

In [6]:
xtra,xtest,ytra,ytest=train_test_split(x,y,test_size=0.2)

In [7]:
xtran=trans.fit_transform(x)

In [8]:
model1=RandomForestRegressor()
model2=LinearRegression()
model3=SVR()
model4=DecisionTreeRegressor()
model5=KNeighborsRegressor()
model6=LGBMRegressor()
model7=xgb.XGBRegressor()

In [9]:
pi1=Pipeline(steps=[('tra',trans),('model1',model1)])
pi2=Pipeline(steps=[('tra',trans),('model2',model2)])
pi3=Pipeline(steps=[('tra',trans),('model3',model3)])
pi4=Pipeline(steps=[('tra',trans),('model4',model4)])
pi5=Pipeline(steps=[('tra',trans),('model5',model5)])
pi6=Pipeline(steps=[('tra',trans),('model6',model6)])
pi7=Pipeline(steps=[('tra',trans),('model7',model7)])

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Results</p>

In [10]:
cross_val_score(pi1,x,y,scoring='r2').mean()

0.862526938939828

In [11]:
cross_val_score(pi2,x,y,scoring='r2')

array([0.8169062 , 0.79325371, 0.83060973, 0.84445932, 0.81583581])

In [12]:
cross_val_score(pi3,x,y,scoring='r2')

array([-0.01087002, -0.01540531, -0.07597663, -0.04040111, -0.02786389])

In [13]:
cross_val_score(pi4,x,y,scoring='r2')

array([0.71448291, 0.69793848, 0.77222369, 0.75995982, 0.75750817])

In [14]:
cross_val_score(pi5,x,y,scoring='r2').mean()

0.8373851178479527

In [15]:
cross_val_score(pi6,x,y,scoring='r2').mean()

0.8807608724234008

In [16]:
cross_val_score(pi7,x,y,scoring='r2').mean()

0.876755958387327

In [17]:
pi6.fit(x,y)

Pipeline(steps=[('tra',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oh',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['name', 'fuel',
                                                   'seller_type',
                                                   'transmission', 'seats',
                                                   'owner', 'Mileage Unit',
                                                   'year']),
                                                 ('ss', StandardScaler(),
                                                  ['km_driven', 'Mileage',
                                                   'Engine (CC)']),
                                                 ('mm', MinMaxScaler(),
                                                  ['km_driven', 'Mileage',
                                                   'Engine (CC)'])])),
                ('model6', LGBMRegressor())])

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Model Saving</p>

In [18]:
import pickle as pk
pk.dump(pi6,open('/kaggle/working/model1.pkl','wb'))

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#4F200D; font-size:140%; text-align:center;padding: 0px; border-bottom: 3px solid #4F200D">Model Utilization</p>

In [19]:
"""import pickle as pk
import pandas as pd
from lightgbm import LGBMRegressor

model = pk.load(open('/kaggle/working/model1.pkl', 'rb'))

def predict_price():
    # Predefined choices for certain inputs
    available_brands = ['Maruti', 'Hyundai', 'Toyota', 'Ford', 'Mahindra', 'Tata', 'Volkswagen', 'Renault', 'Chevrolet', 'Honda']
    available_fuels = ['Diesel', 'Petrol', 'CNG', 'LPG']
    available_seller_types = ['Individual', 'Dealer', 'Trustmark Dealer']
    available_transmissions = ['Manual', 'Automatic']
    available_owners = ['First Owner', 'Second Owner', 'Third Owner', 'Fourth & Above Owner', 'Test Drive Car']
    available_mileage_units = ['kmpl', 'km/kg']

    # Get user input for car details
    name = input(f"Enter the car name {available_brands}: ")
    year = int(input("Enter the manufacturing year (e.g., 2014): "))
    km_driven = int(input("Enter the kilometers driven (range 1000 to 270000): "))
    fuel = input(f"Enter the fuel type {available_fuels}: ")
    seller_type = input(f"Enter the seller type {available_seller_types}: ")
    transmission = input(f"Enter the transmission type {available_transmissions}: ")
    owner = input(f"Enter the owner type {available_owners}: ")
    seats = int(input("Enter the number of seats (e.g., 5): "))
    mileage_unit = input(f"Enter the mileage unit {available_mileage_units}: ")
    mileage = float(input("Enter the mileage (range 19.60 to 25.59): "))
    engine_cc = int(input("Enter the Engine CC (range 624 to 1598): "))

    # Assuming you have the necessary data processing and feature engineering steps here
    # Create a DataFrame with the input data
    data = pd.DataFrame({
        'name': [name],
        'year': [year],
        'km_driven': [km_driven],
        'fuel': [fuel],
        'seller_type': [seller_type],
        'transmission': [transmission],
        'owner': [owner],
        'seats': [seats],
        'Mileage Unit': [mileage_unit],
        'Mileage': [mileage],
        'Engine (CC)': [engine_cc],
        'max_power (in bph)': [0]  # Add a placeholder value for the missing column
    })

    # Assuming you have the necessary preprocessing steps like encoding categorical variables

    # Make predictions using the loaded model
    predicted_price = model.predict(data)

    # Return only the predicted output
    return predicted_price[0]

# Example usage:
result = predict_price()
print(result)"""


'import pickle as pk\nimport pandas as pd\nfrom lightgbm import LGBMRegressor\n\nmodel = pk.load(open(\'/kaggle/working/model1.pkl\', \'rb\'))\n\ndef predict_price():\n    # Predefined choices for certain inputs\n    available_brands = [\'Maruti\', \'Hyundai\', \'Toyota\', \'Ford\', \'Mahindra\', \'Tata\', \'Volkswagen\', \'Renault\', \'Chevrolet\', \'Honda\']\n    available_fuels = [\'Diesel\', \'Petrol\', \'CNG\', \'LPG\']\n    available_seller_types = [\'Individual\', \'Dealer\', \'Trustmark Dealer\']\n    available_transmissions = [\'Manual\', \'Automatic\']\n    available_owners = [\'First Owner\', \'Second Owner\', \'Third Owner\', \'Fourth & Above Owner\', \'Test Drive Car\']\n    available_mileage_units = [\'kmpl\', \'km/kg\']\n\n    # Get user input for car details\n    name = input(f"Enter the car name {available_brands}: ")\n    year = int(input("Enter the manufacturing year (e.g., 2014): "))\n    km_driven = int(input("Enter the kilometers driven (range 1000 to 270000): "))

<p style="font-family: JetBrains Mono; font-weight: bold; letter-spacing: 2px; color: #008080; font-size: 25px; text-align: center; padding: 0px; border-bottom: 3px solid #008080;">Thank u for visiting</p>
